# Song Genre Classification

![](spo1.png)

### Contents
* [**Abstract**](#abstract)
* [**Dataset**](#dataset)
* [**Importing Dataset and the required libraries**](#import)
* [**Information about two datasets**](#info)
* [**Data Preparation for Classifier Model**](#data-prep)
* [**Spliting the Genres in the dataset**](#split)
    * [Method 1](#m1)
    * [Method 2](#m2)
* [**Classifier Model Creation**](#part-b)
    * [Spliting the dataset](#train-test-split)
    * [Deploying the Classifier Models](#deploy)
    * [Cross Validation Score](#cv)
    * [Accuracy Metrices](#accuracy)
* [**Conclusion**](#conclusion)

<a id = 'abstract'></a>
### Abstract

Spotify, with a net worth of $26 billion, is reigning the music streaming platform today. It currently has millions of songs in its database and claims to have the right music score for everyone. Spotify’s Discover Weekly service has become a hit with the millennials. Needless to say, Spotify has invested a lot in research to improve the way users find and listen to music. Machine Learning is at the core of their research. From NLP to Collaborative filtering to Deep Learning, Spotify uses them all. Songs are analyzed based on their digital signatures for some factors, including tempo, acoustics, energy, danceability, etc.

The task at hand is to categorize songs into certain categories (trap, rap, pop etc) based off of given values such as danceability, acousticness and other metrics that can be readily obtained via Spotify's organize playlist.  

Note that both sets of data use the same metrics available off of Spotify's organize playlist feature. These metrics are as follows:
1. **Genre** - the genre of the track
2. **Year** - the release year of the recording. Note that due to vagaries of releases, re-releases, re-issues and general madness, sometimes the release years are not what you'd expect.
3. **Added** - the earliest date you added the track to your collection.
4. **Beats Per Minute** (BPM) - The tempo of the song.
5. **Energy** - The energy of a song - the higher the value, the more energtic. song
6. **Danceability** - The higher the value, the easier it is to dance to this song.
7. **Loudness** (dB) - The higher the value, the louder the song.
8. **Liveness** - The higher the value, the more likely the song is a live recording.
9. **Valence** - The higher the value, the more positive mood for the song.
10. **Length** - The duration of the song.
11. **Acousticness** - The higher the value the more acoustic the song is.
12. **Speechiness** - The higher the value the more spoken word the song contains.
13. **Popularity** - The higher the value the more popular the song is.
14. **Duration** - The length of the song.

<a id = 'dataset'></a>
### Dataset

Here to build this model we are going to use, two datasets so that we have a large amount of data to train and test the classification models.
1. Spotify Mega Dataset :  https://www.kaggle.com/iamsumat/spotify-top-2000s-mega-dataset
2. Spotify 2010-2019 songs Dataset : https://www.kaggle.com/leonardopena/top-spotify-songs-from-20102019-by-year

<a id = 'import'></a>
### Obtaining Data

In [1]:
from pandas import read_csv
import pandas as pd
import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
import numpy as np

In [2]:
df_2000 = pd.read_csv("Spotify-2000.csv")
df_top10s = pd.read_csv("top10s.csv", engine='python')
df_2000.head()

,Index,Title,Artist,Top Genre,Year,Beats Per Minute (BPM),Energy,Danceability,Loudness (dB),Liveness,Valence,Length (Duration),Acousticness,Speechiness,Popularity
0,1,Sunrise,Norah Jones,adult standards,2004,157,30,53,-14,11,68,201,94,3,71
1,2,Black Night,Deep Purple,album rock,2000,135,79,50,-11,17,81,207,17,7,39
2,3,Clint Eastwood,Gorillaz,alternative hip hop,2001,168,69,66,-9,7,52,341,2,17,69
3,4,The Pretender,Foo Fighters,alternative metal,2007,173,96,43,-4,3,37,269,0,4,76
4,5,Waitin' On A Sunny Day,Bruce Springsteen,classic rock,2002,106,82,58,-5,10,87,256,1,3,59


In [3]:
df_top10s.head()

,Unnamed: 0,title,artist,top genre,year,bpm,nrgy,dnce,dB,live,val,dur,acous,spch,pop
0,1,"Hey, Soul Sister",Train,neo mellow,2010,97,89,67,-4,8,80,217,19,4,83
1,2,Love The Way You Lie,Eminem,detroit hip hop,2010,87,93,75,-5,52,64,263,24,23,82
2,3,TiK ToK,Kesha,dance pop,2010,120,84,76,-3,29,71,200,10,14,80
3,4,Bad Romance,Lady Gaga,dance pop,2010,119,92,70,-4,8,71,295,0,4,79
4,5,Just the Way You Are,Bruno Mars,pop,2010,109,84,64,-5,9,43,221,2,4,78


<a id = 'info'></a>
### Information about the two datasets

In [4]:
df_2000.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1994 entries, 0 to 1993
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Index                   1994 non-null   int64 
 1   Title                   1994 non-null   object
 2   Artist                  1994 non-null   object
 3   Top Genre               1994 non-null   object
 4   Year                    1994 non-null   int64 
 5   Beats Per Minute (BPM)  1994 non-null   int64 
 6   Energy                  1994 non-null   int64 
 7   Danceability            1994 non-null   int64 
 8   Loudness (dB)           1994 non-null   int64 
 9   Liveness                1994 non-null   int64 
 10  Valence                 1994 non-null   int64 
 11  Length (Duration)       1994 non-null   object
 12  Acousticness            1994 non-null   int64 
 13  Speechiness             1994 non-null   int64 
 14  Popularity              1994 non-null   int64 
dtypes: i

In [5]:
df_top10s.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 603 entries, 0 to 602
Data columns (total 15 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  603 non-null    int64 
 1   title       603 non-null    object
 2   artist      603 non-null    object
 3   top genre   603 non-null    object
 4   year        603 non-null    int64 
 5   bpm         603 non-null    int64 
 6   nrgy        603 non-null    int64 
 7   dnce        603 non-null    int64 
 8   dB          603 non-null    int64 
 9   live        603 non-null    int64 
 10  val         603 non-null    int64 
 11  dur         603 non-null    int64 
 12  acous       603 non-null    int64 
 13  spch        603 non-null    int64 
 14  pop         603 non-null    int64 
dtypes: int64(12), object(3)
memory usage: 70.8+ KB


In [6]:
len(df_2000["Top Genre"].unique()), len(df_top10s["top genre"].unique())

(149, 50)

In [7]:
df_2000["Top Genre"].value_counts(), df_top10s["top genre"].value_counts()

(album rock                 413
 adult standards            123
 dutch pop                   88
 alternative rock            86
 dance pop                   83
                           ... 
 alaska indie                 1
 alternative country          1
 laboratorio                  1
 alternative pop              1
 german alternative rock      1
 Name: Top Genre, Length: 149, dtype: int64,
 dance pop                    327
 pop                           60
 canadian pop                  34
 barbadian pop                 15
 boy band                      15
 electropop                    13
 british soul                  11
 big room                      10
 neo mellow                     9
 canadian contemporary r&b      9
 art pop                        8
 complextro                     6
 hip pop                        6
 australian dance               6
 atl hip hop                    5
 australian pop                 5
 edm                            5
 latin                   

**Observation**

From a quick observation, we can see certain things about the data we are dealing with:<br>
1. Both datasets use the same metrics, albeit with different column names so we will have to change those names. Fortunately, the same order is kept for both datasets
2. Our target label is the top genre category, however there is a very large number of genre types, so many in fact that our model will be very inaccurate and inefficient if we attempt to label all categories as they are, especially since a large number of categories contain a single song only. 
3. The two datasets do not share the same # of genres.
4. We don't care about the artist, title, index/Unamed: 0, and year. I simply don't think these would have a strong connection with the genre. (Arguably, artist would have a connection to the music genre, but I personally think there is too much variance and it wouldn't be a consistent pattern)

<a id = 'data-prep'></a>
### Data Preparation for the Classifer Model

Dropping off all unecessary columns

In [8]:
df_top10s.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 603 entries, 0 to 602
Data columns (total 15 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  603 non-null    int64 
 1   title       603 non-null    object
 2   artist      603 non-null    object
 3   top genre   603 non-null    object
 4   year        603 non-null    int64 
 5   bpm         603 non-null    int64 
 6   nrgy        603 non-null    int64 
 7   dnce        603 non-null    int64 
 8   dB          603 non-null    int64 
 9   live        603 non-null    int64 
 10  val         603 non-null    int64 
 11  dur         603 non-null    int64 
 12  acous       603 non-null    int64 
 13  spch        603 non-null    int64 
 14  pop         603 non-null    int64 
dtypes: int64(12), object(3)
memory usage: 70.8+ KB


In [9]:
df_2000.drop(columns = ['Index', 'Title', 'Artist', 'Year'], inplace = True)
df_top10s.drop(columns = ['Unnamed: 0', 'title', 'artist', 'year'], inplace = True)

Now both datasets have the same # of columsn in the same order, so we can now join them together after renaming

In [10]:
df_top10s.columns = df_2000.columns # setting column names as each other
df = df_2000.append(df_top10s, ignore_index = True)

both X_train and X_test have several values that are string type numbers, separated with a comma. We have to get rid of these commas then turn them into floats

In [11]:
attributes = df.columns[1:]
for attribute in attributes:
    temp = df[attribute]
    for instance in range(len(temp)):
        if(type(temp[instance]) == str):
            df[attribute][instance] = float(temp[instance].replace(',',''))
# check data types using df.dtype

D:\my project\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


<a id = 'split'></a>
### Splitting Genre in the dataset

Now that we have obtained our full dataset, we need a way to split the top genres. Two methods will be explored: <br>

1. **Method 1** All related songs of a specific category will be placed in a broader category (i.e. celtic pop and indie pop will be placed under the larger theme of pop). The main assumption using this method is that there is minimal difference between differing types of a similar music genre. This will be multiclass classification <br>


2. **Method 2** Songs will have their genres split by space and multilabel classification will be used. <br>

**NOTE**: Method 2 has been pushed to the bottom after Method 1 is completed to prevent interference and distraction from cross coding

In [12]:
# first extracting the genre columns
# getting rid of white spaces and turning it all into lower cases
genre = (df["Top Genre"].str.strip()).str.lower()

<a id = 'm1'></a>
<h4> Method 1

In [13]:
# function to split the genre column
def genre_splitter(genre):
    result = genre.copy()
    result = result.str.split(" ",1)
    for i in range(len(result)):
        if (len(result[i]) > 1):
            result[i] = [result[i][1]]
    return result.str.join('')

<a id = 'm2'></a>
**Method 2**

In [14]:
# loop until the genre cannot be split any further
genre_m1 = genre.copy()
while(max((genre_m1.str.split(" ", 1)).str.len()) > 1):
    genre_m1 = genre_splitter(genre_m1)

In [15]:
len(genre_m1.unique())

73

We reduced our target label by over 50%. Let's take a quick look at our new shortened labels

In [16]:
genre_m1.value_counts()

rock           857
pop            802
standards      123
metal           93
indie           78
              ... 
levenslied       1
ambient          1
laboratorio      1
soundtrack       1
trance           1
Name: Top Genre, Length: 73, dtype: int64

Note that there are certain music genres that have a single value. These genres would make our model inefficient, since it does not have data to work off of, so these values and corresponding rows in the original dataframe will be taken out. Putting all of these instances into a broader "Other" category is another potential solution, but I decided against it because there is probably minimal similarity between all of these music genres. 

In [17]:
unique = genre_m1.unique()
to_remove = [] 

# genres that have a single instance only will be placed within the to_remove array
for genre in unique:
    if genre_m1.value_counts()[genre] < 20: # 10 was arbitrarily chosen
        to_remove += [genre]
len(to_remove)

58

Now replacing our original genre columns with our updated version

In [18]:
df['Top Genre'] = genre_m1
df

,Top Genre,Beats Per Minute (BPM),Energy,Danceability,Loudness (dB),Liveness,Valence,Length (Duration),Acousticness,Speechiness,Popularity
0,standards,157,30,53,-14,11,68,201,94,3,71
1,rock,135,79,50,-11,17,81,207,17,7,39
2,hop,168,69,66,-9,7,52,341,2,17,69
3,metal,173,96,43,-4,3,37,269,0,4,76
4,rock,106,82,58,-5,10,87,256,1,3,59
...,...,...,...,...,...,...,...,...,...,...,...
2592,pop,104,66,61,-7,20,16,176,1,3,75
2593,pop,95,79,75,-6,7,61,206,21,12,75
2594,pop,136,76,53,-5,9,65,260,7,34,70
2595,pop,114,79,60,-6,42,24,217,1,7,69


In [19]:
df.set_index(["Top Genre"],drop = False, inplace = True)
for name in to_remove:
    type(name)
    df.drop(index = str(name), inplace = True)
    

In [20]:
df["Top Genre"].value_counts()

rock         857
pop          802
standards    123
metal         93
indie         78
soul          56
cabaret       51
hop           43
wave          42
invasion      36
europop       27
mellow        26
dance         22
band          21
folk          20
Name: Top Genre, dtype: int64

***************

<a id = 'part-b'></a>
## Classifier Model Creation
A classification model tries to draw some conclusion from the input values given for training. It will predict the class labels/categories for the new data.

Here we are going to prepare several Classification machine learning models based on those we will do a comparative analysis that which model is better among them.

Since we are dealing with a classification problem, several models will be tried:

1. **Random Forest Classifier** : Random Forest is a popular machine learning algorithm that belongs to the supervised learning technique. It can be used for both Classification and Regression problems in ML. It is based on the concept of ensemble learning, which is a process of combining multiple classifiers to solve a complex problem and to improve the performance of the model.


2. **Naive Bayes** : This method is expected to be called several times consecutively on different chunks of a dataset so as to implement out-of-core or online learning. This is especially useful when the whole dataset is too big to fit in memory at once. This method has some performance and numerical stability overhead, hence it is better to call partial_fit on chunks of data that are as large as possible (as long as fitting in the memory budget) to hide the overhead.


3. **Stochastic Gradient Descent Classifier** : Stochastic Gradient Descent (SGD) is a simple yet efficient optimization algorithm used to find the values of parameters/coefficients of functions that minimize a cost function. In other words, it is used for discriminative learning of linear classifiers under convex loss functions such as SVM and Logistic regression.


4. **Logistic Regression** : Logistic regression is a statistical model that in its basic form uses a logistic function to model a binary dependent variable, although many more complex extensions exist. In regression analysis, logistic regression (or logit regression) is estimating the parameters of a logistic model (a form of binary regression).

 
We are going to use these four algorithms and based on the scores of the models the most fitted algorithm will be set! Now let's check out the algorithms

<a id="train-test-split"></a>
**Training and Testing Dataset Spliting using the `train_test_split`**
  
  * Immporting the library from the sklearn.model_selection
  * Split the dataset into 80:20 ratio
  * x_train1 and y_train1 are the trainning datasets
  * x_test1 and y_test1 are the testing datasets
  * After the spliting of the datasets the model is ready to be prepared!

first preparing the training and testing dataset with proper standardization using StandardScaler

In [21]:
train_set, test_set = train_test_split(df, test_size = 0.2, random_state = 42)
# training set
X_train = train_set.values[:,1:]
y_train = train_set.values[:,0]

# test set
X_test = test_set.values[:,1:]
y_test = test_set.values[:,0]

In [22]:
from sklearn.preprocessing import StandardScaler

standard_scaler = StandardScaler().fit(X_train)

# Standard Scaler
X_train_ST = standard_scaler.transform(X_train)
X_test_ST = standard_scaler.transform(X_test)

The labels need to be converted into a form that can be understood by the models, One hot encoding will be used here

In [23]:
# obtaining all unique classes
unique = np.unique(y_train)

In [24]:
from sklearn.preprocessing import label_binarize
from sklearn.preprocessing import LabelEncoder
# 1 hot encoding
y_test_1hot = label_binarize(y_test, classes = unique)
y_train_1hot = label_binarize(y_train, classes = unique)

# labelling
y_test_label = LabelEncoder()

Now creating the instances of the models

<a id = 'deploy'></a>

### Deploying the Classifier Models
We are going to build four models - 
1. Logistic Regression
2. Random Forest Classifier
3. Gaussian Naive Bayes
4. Gradient Descent Classifier

In [25]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier
from sklearn.multiclass import OneVsOneClassifier

models = []
models += [['Naive Bayes', GaussianNB()]]
models += [['SGD', OneVsOneClassifier(SGDClassifier())]]
models += [['Logistic', LogisticRegression(multi_class = 'ovr')]]
rand_forest = RandomForestClassifier(random_state = 42, min_samples_split = 5)

Now training the models using k cross validation

<a id = 'cv'></a>
### Cross Validation Score
Cross-validation is a statistical method used to estimate the skill of machine learning models. That k-fold cross validation is a procedure used to estimate the skill of the model on new data. There are common tactics that you can use to select the value of k for your dataset.

In [26]:
result_ST =[]
kfold = StratifiedKFold(n_splits = 10, random_state = 1, shuffle = True)

# Random Forest has to be done separately since it takes in one hot encoded labels instead
RF_cross_val_score = cross_val_score(rand_forest, X_train_ST, y_train_1hot, cv = 10, scoring = 'accuracy')
print('%s: %f (%f)' % ('Random Forest', RF_cross_val_score.mean(), RF_cross_val_score.std()))

for name, model in models:
    cv_score = cross_val_score(model, X_train_ST, y_train, cv = kfold, scoring = 'accuracy')
    result_ST.append(cv_score)
    print('%s: %f (%f)' % (name,cv_score.mean(), cv_score.std()))

Random Forest: 0.371852 (0.053181)
Naive Bayes: 0.409901 (0.026479)
SGD: 0.491004 (0.040855)
Logistic: 0.560679 (0.026482)


All of our models have very low accuracy, which is most likely due to the lack of available data. Let's move forward however, and select a model. While it appears that Logistic Regression has the highest accuracy (56%), let's calculate the respective recall and precision values. Note that micro averaging will be used for the models (w/ exception to Random Forest) bc of the imbalance in class examples.

<a id = 'accuracy'></a>
### Accuracy Scores using Different Metrices
The accuracy of a machine learning classification algorithm is one way to measure how often the algorithm classifies a data point correctly. Accuracy is the number of correctly predicted data points out of all the data points.

1. **Precision Score** quantifies the number of positive class predictions that actually belong to the positive class.

2. **Recall** quantifies the number of positive class predictions made out of all positive examples in the dataset.

3. **F-Measure** provides a single score that balances both the concerns of precision and recall in one number.

In [27]:
from sklearn.metrics import precision_score, recall_score

result_precision_recall = []

# same reasoning as before for Random Forest
y_temp_randforest = cross_val_predict(rand_forest, X_train_ST, y_train_1hot, cv = 10)
result_precision_recall += [['Random Forest', precision_score(y_train_1hot, y_temp_randforest, average = "micro"), 
                            recall_score(y_train_1hot, y_temp_randforest, average = "micro")]]

print('%s| %s: %f, %s (%f)' % ('Random Forest', 'Precision Score: ', precision_score(y_train_1hot, y_temp_randforest, average = "micro"), 
                           'Recall Score: ', recall_score(y_train_1hot, y_temp_randforest, average = "micro")))

for name, model in models:
    y_pred = cross_val_predict(model, X_train_ST, y_train, cv = kfold)
    precision = precision_score(y_train, y_pred, average = "micro")
    recall = recall_score(y_train, y_pred, average = "micro")
    # storing the precision and recall values
    result_precision_recall += [[name , precision, recall]]
    print('%s| %s: %f, %s (%f)' % (name, 'Precision Score: ', precision, 'Recall Score: ', recall))


Random Forest| Precision Score: : 0.712944, Recall Score:  (0.371802)
Naive Bayes| Precision Score: : 0.409907, Recall Score:  (0.409907)
SGD| Precision Score: : 0.512248, Recall Score:  (0.512248)
Logistic| Precision Score: : 0.560697, Recall Score:  (0.560697)


From the given precision and recall scores, we can now find the respective f1 score and use the highest score to select our model.

In [28]:
from sklearn.metrics import f1_score

for name, precision, recall in result_precision_recall:
    print("%s: %f" % (name, 2 * (precision * recall) / (precision + recall)))

Random Forest: 0.488730
Naive Bayes: 0.409907
SGD: 0.512248
Logistic: 0.560697


Given that Logistic Regression has the highest f1 score, we will move forward with that model.

**************

<a id = 'conclusion'></a>
## Conclusion and Comparison
We have deployed four machine learning algorithms and every algorithm is deployed successfully without any hesitation. We have checked the accuracy of the models based on the accuracy score of each of the models. Now let's take a look at the scores of each models.

|Name of the Model|Cross Validation Score|Precision Score|Recall Score|F-Measure Score|
|:---:|:---:|:---:|:---:|:---:|
|Logistic Regression|0.56|0.56|0.56|0.56|
|Random Forest Classifier|0.37|0.71|0.37|0.49|
|Gradient Descent Algorithm|0.49|0.52|0.51|0.51|
|Gaussian Naive Bayes Algorithm|0.41|0.41|0.41|0.41|

**Comparing all those scores scored by the machine learning algorithms, it is clear that Logistic Regression is having the upper hand in case of this dataset and after this, we can use Gradient Descent Classifier which are also having good score as compared to the other deployed algorithms**

Best Fitted Models ranking - 
1. Logistic Regression
2. Gradient Descent Algorithm
3. Random Forest Classifier
4. Gaussian Naive Bayes Classifier



Hooray!! The models are deployed successfully!


### Hope this project will help you! Thank you!